In [2]:
%matplotlib inline
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models
import os

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
import graduation_pytorch as gra

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchvision import transforms, datasets

In [3]:
# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_augs = transforms.Compose([
        transforms.RandomResizedCrop(size=224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

test_augs = transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        normalize
    ])

In [4]:
train_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/train', transform=train_augs)
test_dataset = datasets.ImageFolder(root='../数据集/UCMerced_LandUse/data/test', transform=test_augs)

In [5]:
train_dataset,test_dataset  # 总：21*100 train:21*80  test：21*20

(Dataset ImageFolder
     Number of datapoints: 1680
     Root Location: ../数据集/UCMerced_LandUse/data/train
     Transforms (if any): Compose(
                              RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                              RandomHorizontalFlip(p=0.5)
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target Transforms (if any): None,
 Dataset ImageFolder
     Number of datapoints: 420
     Root Location: ../数据集/UCMerced_LandUse/data/test
     Transforms (if any): Compose(
                              Resize(size=256, interpolation=PIL.Image.BILINEAR)
                              CenterCrop(size=(224, 224))
                              ToTensor()
                              Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                          )
     Target 

In [6]:
feature, label = train_dataset[450]
label

5

In [8]:
pretrained_net = models.resnet34(pretrained=False)

In [9]:
# 所以我们应该将最后的fc成修改我们需要的输出类别数:  21
pretrained_net.fc = nn.Linear(512, 21)
print(pretrained_net.fc)
#此时，pretrained_net的fc层就被随机初始化了，但是其他层依然保存着预训练得到的参数。

Linear(in_features=512, out_features=21, bias=True)


In [12]:
# output_params = list(map(id, pretrained_net.fc.parameters()))
# feature_params = filter(lambda p: id(p) not in output_params, pretrained_net.parameters()) # 除fc层的参数

lr = 0.01
# optimizer = optim.SGD([{'params': feature_params},
#                        {'params': pretrained_net.fc.parameters(), 'lr': lr * 10}],
#                        lr=lr, weight_decay=0.001)
optimizer = optim.SGD(pretrained_net.parameters(), lr=lr, weight_decay=0.001)

In [13]:
train_iter = DataLoader(train_dataset,
                            batch_size = 64, shuffle=True)
test_iter = DataLoader(test_dataset,
                            batch_size = 64, shuffle=False)

In [14]:
len(train_iter) # 1680/64

27

In [15]:
def train_fine_tuning(net, optimizer, batch_size=32, num_epochs=5):
    loss = torch.nn.CrossEntropyLoss()
    d2l.train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

## 未使用微调

In [16]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 2.9112, train acc 0.093, test acc 0.079, time 72.9 sec
epoch 2, loss 1.3576, train acc 0.165, test acc 0.086, time 28.1 sec
epoch 3, loss 0.8521, train acc 0.234, test acc 0.117, time 27.1 sec
epoch 4, loss 0.5819, train acc 0.279, test acc 0.195, time 28.7 sec
epoch 5, loss 0.4267, train acc 0.337, test acc 0.224, time 28.1 sec
epoch 6, loss 0.3260, train acc 0.383, test acc 0.317, time 28.2 sec
epoch 7, loss 0.2646, train acc 0.404, test acc 0.160, time 46.0 sec
epoch 8, loss 0.2194, train acc 0.433, test acc 0.250, time 28.7 sec
epoch 9, loss 0.1908, train acc 0.461, test acc 0.248, time 27.8 sec
epoch 10, loss 0.1629, train acc 0.469, test acc 0.248, time 28.2 sec
epoch 11, loss 0.1406, train acc 0.504, test acc 0.290, time 28.2 sec
epoch 12, loss 0.1249, train acc 0.517, test acc 0.226, time 27.2 sec
epoch 13, loss 0.1117, train acc 0.534, test acc 0.307, time 27.6 sec
epoch 14, loss 0.1006, train acc 0.552, test acc 0.302, time 29.2 sec
epoch 15, l

In [17]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 1.3209, train acc 0.583, test acc 0.269, time 27.7 sec
epoch 2, loss 0.6458, train acc 0.604, test acc 0.300, time 28.9 sec
epoch 3, loss 0.4012, train acc 0.614, test acc 0.252, time 28.4 sec
epoch 4, loss 0.2906, train acc 0.635, test acc 0.364, time 29.0 sec
epoch 5, loss 0.2243, train acc 0.662, test acc 0.307, time 28.8 sec
epoch 6, loss 0.1838, train acc 0.666, test acc 0.398, time 29.2 sec
epoch 7, loss 0.1504, train acc 0.677, test acc 0.326, time 28.9 sec
epoch 8, loss 0.1288, train acc 0.674, test acc 0.381, time 27.9 sec
epoch 9, loss 0.1096, train acc 0.696, test acc 0.288, time 28.2 sec
epoch 10, loss 0.0999, train acc 0.687, test acc 0.364, time 28.9 sec
epoch 11, loss 0.0843, train acc 0.707, test acc 0.410, time 29.2 sec
epoch 12, loss 0.0760, train acc 0.729, test acc 0.260, time 27.7 sec
epoch 13, loss 0.0700, train acc 0.705, test acc 0.410, time 30.4 sec
epoch 14, loss 0.0630, train acc 0.735, test acc 0.248, time 29.1 sec
epoch 15, l

In [19]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 15)

training on  cuda
epoch 1, loss 0.6034, train acc 0.801, test acc 0.431, time 28.2 sec
epoch 2, loss 0.2864, train acc 0.811, test acc 0.395, time 29.3 sec
epoch 3, loss 0.1717, train acc 0.844, test acc 0.400, time 29.0 sec
epoch 4, loss 0.1526, train acc 0.820, test acc 0.424, time 28.0 sec
epoch 5, loss 0.1071, train acc 0.835, test acc 0.348, time 28.2 sec
epoch 6, loss 0.0890, train acc 0.822, test acc 0.379, time 28.2 sec
epoch 7, loss 0.0796, train acc 0.826, test acc 0.486, time 27.9 sec
epoch 8, loss 0.0593, train acc 0.854, test acc 0.455, time 27.2 sec
epoch 9, loss 0.0538, train acc 0.849, test acc 0.502, time 29.2 sec
epoch 10, loss 0.0474, train acc 0.852, test acc 0.371, time 28.6 sec
epoch 11, loss 0.0454, train acc 0.846, test acc 0.410, time 28.5 sec
epoch 12, loss 0.0449, train acc 0.848, test acc 0.250, time 27.9 sec
epoch 13, loss 0.0464, train acc 0.815, test acc 0.426, time 31.2 sec
epoch 14, loss 0.0340, train acc 0.854, test acc 0.329, time 28.3 sec
epoch 15, l

In [20]:
PATH = "./my_model/UcRemote_resnet34_no_ft.pt"
torch.save(pretrained_net.state_dict(), PATH)

In [13]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 10)

training on  cuda
epoch 1, loss 0.2114, train acc 0.936, test acc 0.910, time 26.9 sec
epoch 2, loss 0.0891, train acc 0.948, test acc 0.902, time 26.1 sec
epoch 3, loss 0.0561, train acc 0.955, test acc 0.836, time 28.0 sec
epoch 4, loss 0.0363, train acc 0.955, test acc 0.900, time 28.1 sec
epoch 5, loss 0.0308, train acc 0.955, test acc 0.693, time 30.2 sec
epoch 6, loss 0.0233, train acc 0.960, test acc 0.914, time 25.5 sec
epoch 7, loss 0.0176, train acc 0.966, test acc 0.919, time 27.3 sec
epoch 8, loss 0.0140, train acc 0.965, test acc 0.893, time 28.0 sec
epoch 9, loss 0.0127, train acc 0.963, test acc 0.864, time 27.9 sec
epoch 10, loss 0.0116, train acc 0.963, test acc 0.864, time 28.8 sec


In [14]:
train_fine_tuning(pretrained_net, optimizer, num_epochs = 10)

training on  cuda
epoch 1, loss 0.0949, train acc 0.973, test acc 0.898, time 25.4 sec
epoch 2, loss 0.0530, train acc 0.970, test acc 0.869, time 24.9 sec
epoch 3, loss 0.0284, train acc 0.975, test acc 0.893, time 24.8 sec
epoch 4, loss 0.0243, train acc 0.978, test acc 0.900, time 25.0 sec
epoch 5, loss 0.0207, train acc 0.966, test acc 0.857, time 26.9 sec
epoch 6, loss 0.0143, train acc 0.976, test acc 0.914, time 25.3 sec
epoch 7, loss 0.0133, train acc 0.974, test acc 0.907, time 25.7 sec
epoch 8, loss 0.0102, train acc 0.976, test acc 0.898, time 25.1 sec
epoch 9, loss 0.0097, train acc 0.975, test acc 0.914, time 26.8 sec
epoch 10, loss 0.0080, train acc 0.978, test acc 0.902, time 25.5 sec


In [15]:
pretrained_net.fc = nn.Sequential()
PATH = "./my_model/UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)

In [3]:
## 加载模型：resnet34_whurisi_remote_finetune_
pretrained_net = models.resnet34(pretrained=True)
pretrained_net.fc = nn.Linear(512, 21)
PATH = "./my_model/UcRemote_resnet34_fine-tune.pt"
pretrained_net.fc = nn.Sequential()
pretrained_net.load_state_dict(torch.load(PATH))

In [4]:
pretrained_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

## 将平均池化改为最大池化

In [5]:
pretrained_net.avgpool

AvgPool2d(kernel_size=7, stride=1, padding=0)

In [7]:
maxP = nn.MaxPool2d(kernel_size=7, stride=1, padding=0)
maxP

MaxPool2d(kernel_size=7, stride=1, padding=0, dilation=1, ceil_mode=False)

In [8]:
pretrained_net.avgpool = nn.MaxPool2d(kernel_size=7, stride=1, padding=0)

In [11]:
pretrained_net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [10]:
PATH = "./my_model/maxPooling_UcRemote_resnet34_fine-tune.pt"
torch.save(pretrained_net.state_dict(), PATH)